In [39]:
import sys; sys.argv=['']; del sys
import os
import cv2
import sys
import random
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt
import copy
import gzip
import pickle
import h5py

import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from data import load_data, prepare_input, normalize, denormalize
from models import DPINet
from utils import calc_box_init_FluidShake
from copy import deepcopy

In [12]:
parser = argparse.ArgumentParser()
parser.add_argument('--pstep', type=int, default=2)
parser.add_argument('--epoch', type=int, default=0)
parser.add_argument('--iter', type=int, default=0)
parser.add_argument('--env', default='FluidShake')
parser.add_argument('--time_step', type=int, default=0)
parser.add_argument('--time_step_clip', type=int, default=0)
parser.add_argument('--dt', type=float, default=1./60.)
parser.add_argument('--nf_relation', type=int, default=300)
parser.add_argument('--nf_particle', type=int, default=200)
parser.add_argument('--nf_effect', type=int, default=200)
parser.add_argument('--outf', default='files')
parser.add_argument('--dataf', default='data/small/fluid_shake/')
parser.add_argument('--evalf', default='eval')
parser.add_argument('--eval', type=int, default=1)
parser.add_argument('--verbose_data', type=int, default=0)
parser.add_argument('--verbose_model', type=int, default=0)

parser.add_argument('--debug', type=int, default=0)

parser.add_argument('--n_instances', type=int, default=0)
parser.add_argument('--n_stages', type=int, default=0)
parser.add_argument('--n_his', type=int, default=0)

# shape state:
# [x, y, z, x_last, y_last, z_last, quat(4), quat_last(4)]
parser.add_argument('--shape_state_dim', type=int, default=14)

# object attributes:
parser.add_argument('--attr_dim', type=int, default=0)

# object state:
parser.add_argument('--state_dim', type=int, default=0)
parser.add_argument('--position_dim', type=int, default=0)

# relation attr:
parser.add_argument('--relation_dim', type=int, default=0)

args = parser.parse_args()


phases_dict = dict()

if args.env == 'FluidFall':
    env_idx = 4
    data_names = ['positions', 'velocities']

    # object states:
    # [x, y, z, xdot, ydot, zdot]
    args.state_dim = 6
    args.position_dim = 3

    # object attr:
    # [fluid]
    args.attr_dim = 1

    # relation attr:
    # [none]
    args.relation_dim = 1

    args.time_step = 121
    args.time_step_clip = 5
    args.n_instance = 1
    args.n_stages = 1

    args.neighbor_radius = 0.08

    phases_dict["instance_idx"] = [0, 189]
    phases_dict["root_num"] = [[]]
    phases_dict["instance"] = ['fluid']
    phases_dict["material"] = ['fluid']

    args.outf = 'dump_FluidFall/' + args.outf
    args.evalf = 'dump_FluidFall/' + args.evalf

elif args.env == 'BoxBath':
    env_idx = 1
    data_names = ['positions', 'velocities', 'clusters']

    # object states:
    # [x, y, z, xdot, ydot, zdot]
    args.state_dim = 6
    args.position_dim = 3

    # object attr:
    # [rigid, fluid, root_0]
    args.attr_dim = 3

    # relation attr:
    # [none]
    args.relation_dim = 1

    args.time_step = 151
    args.time_step_clip = 0
    args.n_instance = 2
    args.n_stages = 4

    args.neighbor_radius = 0.08

    # ball, fluid
    phases_dict["instance_idx"] = [0, 64, 1024]
    phases_dict["root_num"] = [[8], []]
    phases_dict["root_sib_radius"] = [[0.4], []]
    phases_dict["root_des_radius"] = [[0.08], []]
    phases_dict["root_pstep"] = [[args.pstep], []]
    phases_dict["instance"] = ['cube', 'fluid']
    phases_dict["material"] = ['rigid', 'fluid']

    args.outf = 'dump_BoxBath/' + args.outf
    args.evalf = 'dump_BoxBath/' + args.evalf

elif args.env == 'FluidShake':
    env_idx = 6
    data_names = ['positions', 'velocities', 'shape_quats', 'scene_params']

    height = 1.0
    border = 0.025

    # object states:
    # [x, y, z, xdot, ydot, zdot]
    args.state_dim = 6
    args.position_dim = 3

    # object attr:
    # [fluid, wall_0, wall_1, wall_2, wall_3, wall_4]
    # wall_0: floor
    # wall_1: left
    # wall_2: right
    # wall_3: back
    # wall_4: front
    args.attr_dim = 6

    # relation attr:
    # [none]
    args.relation_dim = 1

    args.n_instance = 1
    args.time_step = 301
    args.time_step_clip = 0
    args.n_stages = 2

    args.neighbor_radius = 0.08

    phases_dict["root_num"] = [[]]
    phases_dict["instance"] = ["fluid"]
    phases_dict["material"] = ["fluid"]

    args.outf = 'dump_FluidShake/' + args.outf
    args.evalf = 'dump_FluidShake/' + args.evalf

elif args.env == 'RiceGrip':
    env_idx = 5
    data_names = ['positions', 'velocities', 'shape_quats', 'clusters', 'scene_params']

    args.n_his = 3

    # object state:
    # [rest_x, rest_y, rest_z, rest_xdot, rest_ydot, rest_zdot,
    #  x, y, z, xdot, ydot, zdot, quat.x, quat.y, quat.z, quat.w]
    args.state_dim = 16 + 6 * args.n_his
    args.position_dim = 6

    # object attr:
    # [fluid, root, gripper_0, gripper_1,
    #  clusterStiffness, clusterPlasticThreshold, clusterPlasticCreep]
    args.attr_dim = 7

    # relation attr:
    # [none]
    args.relation_dim = 1

    args.n_instance = 1
    args.time_step = 41
    args.time_step_clip = 0
    args.n_stages = 4

    args.neighbor_radius = 0.08
    args.n_roots = 30

    phases_dict["root_num"] = [[args.n_roots]]
    phases_dict["root_sib_radius"] = [[5.0]]
    phases_dict["root_des_radius"] = [[0.2]]
    phases_dict["root_pstep"] = [[args.pstep]]
    phases_dict["instance"] = ["rice"]
    phases_dict["material"] = ["fluid"]

    args.outf = 'dump_RiceGrip/' + args.outf
    args.evalf = 'dump_RiceGrip/' + args.evalf

else:
    raise AssertionError("Unsupported env")


args.outf = args.outf + '_' + args.env
args.evalf = args.evalf + '_' + args.env
# args.dataf = 'data/' + args.dataf + '_' + args.env

print(args)


print("Loading stored stat from %s" % args.dataf)
stat_path = os.path.join(args.dataf, 'stat.h5')
stat = load_data(data_names[:2], stat_path)
for i in range(len(stat)):
    stat[i] = stat[i][-args.position_dim:, :]
    # print(data_names[i], stat[i].shape)

Namespace(attr_dim=6, dataf='data/small/fluid_shake/', debug=0, dt=0.016666666666666666, env='FluidShake', epoch=0, eval=1, evalf='dump_FluidShake/eval_FluidShake', iter=0, n_his=0, n_instance=1, n_instances=0, n_stages=2, neighbor_radius=0.08, nf_effect=200, nf_particle=200, nf_relation=300, outf='dump_FluidShake/files_FluidShake', position_dim=3, pstep=2, relation_dim=1, shape_state_dim=14, state_dim=6, time_step=301, time_step_clip=0, verbose_data=0, verbose_model=0)
Loading stored stat from data/small/fluid_shake/


In [49]:
use_gpu = torch.cuda.is_available()

model = DPINet(args, stat, phases_dict, residual=True, use_gpu=use_gpu)

model_file = './dump_FluidShake/files_FluidShake/net_epoch_4_iter_500000.pth'
print("Loading network from %s" % model_file)
model.load_state_dict(torch.load(model_file))
model.eval()

Loading network from ./dump_FluidShake/files_FluidShake/net_epoch_4_iter_500000.pth


DPINet(
  (particle_encoder_list): ModuleList(
    (0): ParticleEncoder(
      (model): Sequential(
        (0): Linear(in_features=18, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=200, bias=True)
        (3): ReLU()
      )
    )
    (1): ParticleEncoder(
      (model): Sequential(
        (0): Linear(in_features=18, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=200, bias=True)
        (3): ReLU()
      )
    )
  )
  (relation_encoder_list): ModuleList(
    (0): RelationEncoder(
      (model): Sequential(
        (0): Linear(in_features=37, out_features=300, bias=True)
        (1): ReLU()
        (2): Linear(in_features=300, out_features=300, bias=True)
        (3): ReLU()
        (4): Linear(in_features=300, out_features=300, bias=True)
        (5): ReLU()
      )
    )
    (1): RelationEncoder(
      (model): Sequential(
        (0): Linear(in_features=37, out_features=300, bia

In [116]:
pruning_perc = 99

# Calculate threshold
num_params_1 = 0
all_parameters = []
for param in model.parameters():
    if param.requires_grad and len(param.data.size()) != 1:  # trainable, not bias
        all_parameters += list(param.cpu().data.abs().numpy().flatten())
        num_params_1 += param.numel()
threshold = np.percentile(np.array(all_parameters), pruning_perc)
print("Threshold: {}".format(threshold))

# Copy parameters
original_params = deepcopy(model.state_dict())
pruned_params = deepcopy(model.state_dict())

# Generate pruned params
num_params_2 = 0
for _, param in pruned_params.items():
    if len(param.data.size()) != 1:  # not bias
        param *= (param.data.abs() > threshold).float()
        num_params_2 += param.numel()
        
assert num_params_1 == num_params_2

Threshold: 0.004021701518466418


In [117]:
model.load_state_dict(pruned_params)
for param in model.parameters():
    print(param)
    
torch.save(model.state_dict(), './dump_FluidShake/files_FluidShake/net_epoch_4_iter_500000_pruning_{}.pth'.format(pruning_perc))

Parameter containing:
tensor([[-0., -0., 0.,  ..., 0., -0., 0.],
        [0., -0., 0.,  ..., 0., 0., 0.],
        [-0., 0., -0.,  ..., 0., -0., -0.],
        ...,
        [-0., -0., -0.,  ..., 0., -0., 0.],
        [-0., -0., 0.,  ..., -0., 0., 0.],
        [-0., -0., -0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([-0.1006,  0.2226,  0.0247, -0.0799, -0.0624, -0.1573,  0.1069, -0.3494,
        -0.0556,  0.1588,  0.0328, -0.1072,  0.0257, -0.0976, -0.4395,  0.1576,
         0.1419, -0.2018,  0.2023, -0.0319, -0.0793, -0.1627,  0.0159,  0.1035,
        -0.3764, -0.0794, -0.2716, -0.1764, -0.2571, -0.2251, -0.0946, -0.2154,
        -0.2689, -0.2232,  0.0837,  0.0248,  0.0238, -0.2525, -0.2887, -0.3432,
        -0.3467, -0.2160, -0.0764, -0.1334,  0.1929, -0.2827,  0.1317,  0.1076,
         0.2466, -0.0193, -0.0741, -0.1230,  0.0007, -0.0326, -0.1787, -0.0700,
        -0.3708, -0.1873, -0.2114, -0.4152,  0.0405, -0.3571,  0.0543,  0.0807,
        -0.3821, -0.1845

In [79]:
use_gpu = torch.cuda.is_available()

model = DPINet(args, stat, phases_dict, residual=True, use_gpu=use_gpu)

model_file = './dump_FluidShake/files_FluidShake/net_epoch_4_iter_500000.pth'
print("Loading network from %s" % model_file)
model.load_state_dict(torch.load(model_file))
model.eval()

Loading network from ./dump_FluidShake/files_FluidShake/net_epoch_4_iter_500000.pth


DPINet(
  (particle_encoder_list): ModuleList(
    (0): ParticleEncoder(
      (model): Sequential(
        (0): Linear(in_features=18, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=200, bias=True)
        (3): ReLU()
      )
    )
    (1): ParticleEncoder(
      (model): Sequential(
        (0): Linear(in_features=18, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=200, bias=True)
        (3): ReLU()
      )
    )
  )
  (relation_encoder_list): ModuleList(
    (0): RelationEncoder(
      (model): Sequential(
        (0): Linear(in_features=37, out_features=300, bias=True)
        (1): ReLU()
        (2): Linear(in_features=300, out_features=300, bias=True)
        (3): ReLU()
        (4): Linear(in_features=300, out_features=300, bias=True)
        (5): ReLU()
      )
    )
    (1): RelationEncoder(
      (model): Sequential(
        (0): Linear(in_features=37, out_features=300, bia

In [80]:
num_params = 0
num_zeros = 0
for param in model.parameters():
    if param.requires_grad:
        num_params += param.numel()
        num_zeros += np.sum((param == 0).numpy())

In [81]:
print(num_params)
print(num_zeros)
print(num_zeros / num_params)

1075610
0.0
0.0
